# Getting Started with Radar DataTree

This notebook demonstrates how to open, explore, and work with radar data stored in the DataTree format.

## Overview

Radar data is naturally hierarchical - a radar volume consists of multiple sweeps at different elevation angles, each containing various data products (reflectivity, velocity, etc.). The DataTree structure from `xarray-datatree` provides an elegant way to represent this hierarchy while maintaining full compatibility with the xarray ecosystem.

## Setup

First, let's import the necessary libraries:

In [ ]:
import xarray as xr
from datatree import DataTree, open_datatree
import numpy as np
import matplotlib.pyplot as plt

## Opening a Radar DataTree

Radar data stored in Zarr format can be opened directly using `open_datatree`:

In [ ]:
# Example: Opening a radar datatree from a Zarr store
# Replace with actual path to your data
# dt = open_datatree("s3://bucket/path/to/radar.zarr", engine="zarr")

# For demonstration, let's create a simple example datatree
# This mimics the structure of actual radar data

# Create sample data for demonstration
n_azimuth = 360
n_range = 500

azimuth = np.linspace(0, 360, n_azimuth, endpoint=False)
range_m = np.linspace(0, 150000, n_range)  # 150 km range

# Create sample datasets for different sweeps
sweeps = {}
for i, elevation in enumerate([0.5, 1.5, 2.5, 3.5, 4.5]):
    # Simulate reflectivity data with some structure
    dbz = np.random.randn(n_azimuth, n_range) * 10 + 20
    vel = np.random.randn(n_azimuth, n_range) * 5
    
    ds = xr.Dataset(
        {
            "DBZ": (["azimuth", "range"], dbz, {"units": "dBZ", "long_name": "Reflectivity"}),
            "VEL": (["azimuth", "range"], vel, {"units": "m/s", "long_name": "Radial Velocity"}),
        },
        coords={
            "azimuth": (["azimuth"], azimuth, {"units": "degrees"}),
            "range": (["range"], range_m, {"units": "meters"}),
        },
        attrs={
            "elevation": elevation,
            "sweep_number": i,
        }
    )
    sweeps[f"sweep_{i}"] = ds

# Create the datatree
dt = DataTree.from_dict(sweeps)
dt.attrs = {
    "instrument_name": "EXAMPLE_RADAR",
    "latitude": 40.0,
    "longitude": -88.0,
    "altitude": 200.0,
}

print("Radar DataTree created successfully!")

## Exploring the DataTree Structure

Let's examine the structure of our radar datatree:

In [ ]:
# Print the tree structure
print(dt)

In [ ]:
# Access root attributes (radar metadata)
print("Radar Metadata:")
for key, value in dt.attrs.items():
    print(f"  {key}: {value}")

In [ ]:
# List all sweeps
print("Available sweeps:")
for child in dt.children:
    sweep = dt[child]
    print(f"  {child}: elevation = {sweep.attrs.get('elevation', 'N/A')}°")

## Accessing Individual Sweeps

Each sweep can be accessed as a standard xarray Dataset:

In [ ]:
# Access the first sweep
sweep_0 = dt["sweep_0"].ds
sweep_0

In [ ]:
# Access specific variables
dbz = sweep_0["DBZ"]
print(f"Reflectivity shape: {dbz.shape}")
print(f"Reflectivity range: {float(dbz.min()):.1f} to {float(dbz.max()):.1f} dBZ")

## Basic Visualization

Let's create a simple plot of the radar data:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot reflectivity
sweep_0["DBZ"].plot(ax=axes[0], cmap="pyart_NWSRef", vmin=-10, vmax=60)
axes[0].set_title("Reflectivity (DBZ)")

# Plot velocity
sweep_0["VEL"].plot(ax=axes[1], cmap="pyart_NWSVel", vmin=-20, vmax=20)
axes[1].set_title("Radial Velocity (VEL)")

plt.tight_layout()
plt.show()

## Working with Multiple Sweeps

One of the advantages of DataTree is the ability to work with multiple sweeps efficiently:

In [ ]:
# Calculate statistics across all sweeps
print("Statistics by sweep:")
print("-" * 50)

for sweep_name in dt.children:
    sweep = dt[sweep_name].ds
    mean_dbz = float(sweep["DBZ"].mean())
    max_dbz = float(sweep["DBZ"].max())
    elevation = sweep.attrs.get("elevation", "N/A")
    print(f"{sweep_name} (elev={elevation}°): mean DBZ = {mean_dbz:.1f}, max DBZ = {max_dbz:.1f}")

## Saving and Loading

DataTrees can be saved to and loaded from Zarr format:

In [ ]:
# Save to Zarr
# dt.to_zarr("example_radar.zarr", mode="w")

# Load from Zarr
# dt_loaded = open_datatree("example_radar.zarr", engine="zarr")

print("To save: dt.to_zarr('path/to/output.zarr', mode='w')")
print("To load: open_datatree('path/to/output.zarr', engine='zarr')")

## Next Steps

- See `02_visualization.ipynb` for advanced visualization with cartopy
- See `03_multi_sweep_analysis.ipynb` for working with radar volumes
- Explore more examples at the [Radar DataTree Demo Repository](https://github.com/earth-mover/radar-data-demo)

## Data Conversion

If you have raw radar data (NEXRAD Level II, ODIM H5, etc.) that you'd like to convert to this format, please contact [@aladinor](https://github.com/aladinor).

---

*For more information, see the paper: [Radar DataTree: A FAIR and Cloud-Native Framework for Scalable Weather Radar Archives](https://doi.org/10.48550/arXiv.2510.24943) (Ladino-Rincón & Nesbitt, 2025)*